In [ ]:
import math as m
import numpy as np
import sympy as sym
import random as rand
import matplotlib.pyplot as plt
import cmath as cm
from tqdm import tqdm
import torch
import itertools
import cmath as cm

In [ ]:
# defininf the states
# defininsg coefficeients sybols

def Creating_states(coeff, abstract = False):    # coeff list like [a0,a1,b0,b1]
    if abstract == True:
        a0 = sym.symbols('a0')
        a1 = sym.symbols('a1')
        b0 = sym.symbols('b0')
        b1 = sym.symbols('b1')
    else:
        a0 = coeff[0]
        a1 = coeff[1]
        b0 = coeff[2]
        b1 = coeff[3]
    psi0 = [a0,a1]      # defining states
    psi1 = [b0,b1]
    return([psi0,psi1])

In [ ]:
def roundc(c, digits):
    if c.imag == 0:
        return round(c.real, digits)
    else:
        return round(c.real, digits) + round(c.imag, digits) * 1j

In [ ]:
def update_lr(lr, i):
    return lr/(i+1)

In [ ]:
def fnj_vec(N, coeff, priors):

    # creating the SIC POVM matrices
    w = torch.exp((2/3)*torch.complex(torch.tensor([0.0]), torch.tensor([1.0]))*m.pi)
    POVM_vec = (1/(2**.5))*(torch.tensor([[0,1,-1],[-1,0,1],[1,-1,0],[0,w,-w**2],[-1,0,w**2],[1,-w,0],[0,w**2,-w],[-1,0,w],[1,-w**2,0]], dtype=torch.cdouble))  # an array of POVM direction vectors
    POVM_elts = [(1/3)*torch.outer(torch.conj(POVM_vec[i]),POVM_vec[i]) for i in range(len(POVM_vec))]   # a list of POVM matrix

    initial_states = Creating_states(coeff = coeff, abstract=False)     # Creating the two states with these coefficients
    psi0 = initial_states[0]
    psi1 = initial_states[1]    # created the states to be discriminated
    
    psi0sq = []
    psi1sq = []
    [[psi0sq.append(i*j) for i in psi0] for j in psi0]
    [[psi1sq.append(i*j) for i in psi1] for j in psi1]   
    psi0psi0 = [psi0sq[0], np.sqrt(psi0sq[1]**2+psi0sq[2]**2), psi0sq[3]]    
    psi1psi1 = [psi1sq[0], np.sqrt(psi1sq[1]**(2)+psi1sq[2]**(2)), psi1sq[3]]    # creating square states
    
    vec_psi0psi0 = np.array(psi0psi0)     
    vec_psi1psi1 = np.array(psi1psi1)
    rho = priors[0]*np.outer(vec_psi0psi0, vec_psi0psi0)+ priors[1]*np.outer(vec_psi1psi1, vec_psi1psi1)     # theoretical density matrix with priors 1/2 each.

    prob_vec =  [np.trace(np.dot(POVM_elts[i],rho)) for i in range(9)] 
    prob_vec = [i.real for i in prob_vec if abs(i.imag) < .01]          # cleaned up theoretical prob vector

    POVM_dir_symbols = ['d1','d2','d3','d4','d5','d6','d7','d8','d9']      # symbols to indicate collapsed direction
    #prob distribution is simply the corresponding elements of the prob_vec
    collapse_dir_vec = rand.choices(POVM_dir_symbols, weights=prob_vec, k = N)   # choosing collapse directions with weights for N trials

    nj_vec = [collapse_dir_vec.count(f'd{i+1}') for i in range(9)]

    return nj_vec

In [ ]:
def torch_out(a, f0, f1, t0, t1):

    y_pred = torch.empty(9)
    
    w0 = (torch.cos(a))**2
    w1 = (torch.sin(a))**2
    a0 = torch.cos(t0)
    b0 = torch.sin(t0)
    a1 = torch.cos(t1)
    b1 = torch.sin(t1)

    n2 = torch.tensor([2.0])
    
    y_pred[0] = (1/6)*(w0*(b0**2)*(1 + a0**2 - 2*torch.sqrt(n2)*torch.cos(f0)*a0*b0) + w1*(b1**2)*(1 + a1**2 - 2*torch.sqrt(n2)*torch.cos(f1)*a1*b1)) #p1
    y_pred[1] = (1/6)*(w0*(a0**2)*(1 + b0**2 - 2*torch.sqrt(n2)*torch.cos(f0)*a0*b0) + w1*(a1**2)*(1 + b1**2 - 2*torch.sqrt(n2)*torch.cos(f1)*a1*b1)) #p2
    y_pred[2] = (1/6)*(w0*(a0**4 + b0**4 - 2*torch.cos(2*f0)*(a0**2)*(b0**2)) + w1*(a1**4 + b1**4 - 2*torch.cos(2*f1)*(a1**2)*(b1**2))) #p3
    y_pred[3] = (1/6)*(w0*(b0**2)*(1 + a0**2 - 2*torch.sqrt(n2)*torch.cos(f0 + 2*torch.pi/3)*a0*b0) + w1*(b1**2)*(1 + a1**2 - 2*torch.sqrt(n2)*torch.cos(f1 + 2*torch.pi/3)*a1*b1)) #p4
    y_pred[4] = (1/6)*(w0*(a0**2)*(1 + b0**2 - 2*torch.sqrt(n2)*torch.cos(f0 - 4*torch.pi/3)*a0*b0) + w1*(a1**2)*(1 + b1**2 - 2*torch.sqrt(n2)*torch.cos(f1 + 2*torch.pi/3)*a1*b1)) #p5
    y_pred[5] = (1/6)*(w0*(a0**4 + b0**4 - 2*torch.cos(2*f0 - 2*torch.pi/3)*(a0**2)*(b0**2)) + w1*(a1**4 + b1**4 - 2*torch.cos(2*f1 - 2*torch.pi/3)*(a1**2)*(b1**2))) #p6
    y_pred[6] = (1/6)*(w0*(b0**2)*(1 + a0**2 - 2*torch.sqrt(n2)*torch.cos(f0 - 2*torch.pi/3)*a0*b0) + w1*(b1**2)*(1 + a1**2 - 2*torch.sqrt(n2)*torch.cos(f1 - 2*torch.pi/3)*a1*b1)) #p7
    y_pred[7] = (1/6)*(w0*(a0**2)*(1 + b0**2 - 2*torch.sqrt(n2)*torch.cos(f0 - 2*torch.pi/3)*a0*b0) + w1*(a1**2)*(1 + b1**2 - 2*torch.sqrt(n2)*torch.cos(f1 - 2*torch.pi/3)*a1*b1)) #p8
    y_pred[8] = (1/6)*(w0*(a0**4 + b0**4 - 2*torch.cos(2*f0 - 4*torch.pi/3)*(a0**2)*(b0**2)) + w1*(a1**4 + b1**4 - 2*torch.cos(2*f1 + 2*torch.pi/3)*(a1**2)*(b1**2))) #p9

    return y_pred

In [ ]:
def floss(ny, a, f0, f1, t0,  t1):
    y_pred = torch_out(a, f0, f1, t0, t1)
    total_ny = torch.sum(ny)
    log_likelyhood = -(1/total_ny)*torch.sum(ny * torch.log(y_pred))
    return log_likelyhood

In [ ]:
# Loss vs alpha

coeff = [0**.5,1**.5,1**.5,0**.5] 
priors = [.5,.5]
N = 1000

nj_vec = fnj_vec(N, coeff, priors)
    
nj_vec = torch.tensor(nj_vec, dtype = torch.int64)

# Define your fixed values for f0, f1, t0, t1
f0_fixed = torch.tensor([torch.pi / 2])
f1_fixed = torch.tensor([torch.pi / 4])
t0_fixed = torch.tensor([torch.pi / 4])
t1_fixed = torch.tensor([torch.pi / 2])

# Define the range of values for a
a_values = torch.linspace(0, 2 * np.pi, 100)

# Compute torch_out for each value of a
torch_out_values = [torch_out(a,f0_fixed,f1_fixed,t0_fixed,t1_fixed) for a in a_values]
floss_values = [floss(nj_vec, a, f0_fixed, f1_fixed, t0_fixed, t1_fixed) for a in a_values]

# Plot the dependence of torch_out on a
plt.plot(a_values, floss_values)
plt.xlabel('a')
plt.ylabel('floss')
plt.show()

In [ ]:
# Loss vs f0

coeff = [0**.5,1**.5,1**.5,0**.5] 
priors = [.5,.5]
N = 1000

nj_vec = fnj_vec(N, coeff, priors)
    
nj_vec = torch.tensor(nj_vec, dtype = torch.int64)

# Define your fixed values for a, f1, t0, t1
a_fixed = torch.tensor([torch.pi / 4])
f1_fixed = torch.tensor([torch.pi / 4])
t0_fixed = torch.tensor([torch.pi / 4])
t1_fixed = torch.tensor([torch.pi / 4])

# Define the range of values for f0
f0_values = torch.linspace(0, 2 * np.pi, 100)

# Compute torch_out for each value of f0
torch_out_values = [torch_out(a_fixed, f0, f1_fixed, t0_fixed, t1_fixed) for f0 in f0_values]
floss_values = [floss(nj_vec, a_fixed, f0, f1_fixed, t0_fixed, t1_fixed) for f0 in f0_values]

# Plot the dependence of torch_out on f0
plt.plot(a_values, floss_values)
plt.xlabel('f0')
plt.ylabel('floss')
plt.show()

In [ ]:
# Loss vs t0

coeff = [0**.5,1**.5,1**.5,0**.5] 
priors = [.5,.5]
N = 1000

nj_vec = fnj_vec(N, coeff, priors)
    
nj_vec = torch.tensor(nj_vec, dtype = torch.int64)

# Define your fixed values for a, f0, f1, t1
a_fixed = torch.tensor([torch.pi / 4])
f0_fixed = torch.tensor([torch.pi / 4])
f1_fixed = torch.tensor([torch.pi / 4])
t1_fixed = torch.tensor([torch.pi / 4])

# Define the range of values for t0
t0_values = torch.linspace(0, 2 * np.pi, 100)

# Compute torch_out for each value of t0
torch_out_values = [torch_out(a_fixed, f0_fixed, f1_fixed, t0, t1_fixed) for t0 in t0_values]
floss_values = [floss(nj_vec, a_fixed, f0_fixed, f1_fixed, t0, t1_fixed) for t0 in t0_values]

# Plot the dependence of torch_out on t0
plt.plot(a_values, floss_values)
plt.xlabel('t0')
plt.ylabel('floss')
plt.show()

In [ ]:
def grad_fid(learning_rate, N, coeff, priors):
    
    nj_vec = fnj_vec(N, coeff, priors)

    ##################################### GRaDIENT DESCENT ##############################################
    
    nj_vec = torch.tensor(nj_vec, dtype = torch.int64)
    
    # Initialize parameters

    # Define the intervals
    #I_a = [0, np.pi]
    #I_f0 = [0, 2*np.pi]
    #I_f1 = [0, 2*np.pi]
    #I_t0 = [0, 2*np.pi]
    #I_t1 = [0, 2*np.pi]

    # Divide the intervals and generate the values
    #a_values = np.linspace(*I_a, 3)[1:-1]
    a_values = [np.pi/4]
    #f0_values = np.linspace(*I_f0, 6)[1:-1]
    f0_values = np.random.uniform(0.0, np.pi/2, 3).tolist()
    #f1_values = np.linspace(*I_f1, 6)[1:-1]
    f1_values = np.random.uniform(0.0, np.pi/2, 3).tolist()
    #t0_values = np.linspace(*I_t0, 6)[1:-1]
    t0_values = np.random.uniform(0.0, np.pi/2, 3).tolist()
    #t1_values = np.linspace(*I_t1, 6)[1:-1]
    t1_values = np.random.uniform(0.0, np.pi/2, 3).tolist()

    # Generate all possible combinations of the values
    #initial_conditions = [[torch.tensor([val], requires_grad=True) for val in combination] 
                        #for combination in itertools.product(a_values, f0_values, f1_values, t0_values, t1_values)]

    initial_conditions = [(torch.tensor([i], requires_grad=True), torch.tensor([j], requires_grad=True),torch.tensor([k], requires_grad=True),torch.tensor([l], requires_grad=True),torch.tensor([m], requires_grad=True)) for i in a_values for j in f0_values for k in f1_values for l in t0_values for m in t1_values]


    #print(f'initial conditions: {initial_conditions}')

    '''# Define the intervals
    I_a = [0, torch.pi]
    I_f0 = [0, 2*torch.pi]
    I_f1 = [0, 2*torch.pi]
    I_t0 = [0, 2*torch.pi]
    I_t1 = [0, 2*torch.pi]

    # Generate 10 different initial conditions
    initial_conditions = []
    for _ in range(10):
        a = torch.tensor([rand.uniform(*I_a)], requires_grad=True)
        f0 = torch.tensor([rand.uniform(*I_f0)], requires_grad=True)
        f1 = torch.tensor([rand.uniform(*I_f1)], requires_grad=True)
        t0 = torch.tensor([rand.uniform(*I_t0)], requires_grad=True)
        t1 = torch.tensor([rand.uniform(*I_t1)], requires_grad=True)
        initial_conditions.append([a, f0, f1, t0, t1])'''

    '''initial_conditions = [
    [torch.tensor([val], requires_grad=True) for val in [torch.pi / 4, 0.0, 0.0, torch.pi / 2, 0.0]],
    #[torch.tensor([val], requires_grad=True) for val in [torch.pi / 2, 0.0, 0.0, torch.pi / 2, 0.0]],
    # Add more initial conditions here
    ]'''

    '''# Define your initial conditions
    values = [0.0, torch.pi / 4, torch.pi / 2, torch.pi]

    # Generate all possible 5-dimensional combinations of the values
    initial_conditions = [[torch.tensor([val], requires_grad=True) for val in tqdm(combination)] for combination in itertools.product(values, repeat=5)]
    ip = [torch.tensor([torch.pi / 4], requires_grad=True),torch.tensor([0.0], requires_grad=True),torch.tensor([0.0], requires_grad=True),torch.tensor([torch.pi / 2], requires_grad=True),torch.tensor([0.0], requires_grad=True)]
    a = ip[0]#torch.tensor([torch.pi / 4], requires_grad=True) #in_par[0]
    f0 = ip[1]#torch.tensor([0.0], requires_grad=True) #in_par[1]
    f1 = ip[2]#torch.tensor([0.0], requires_grad=True) #in_par[2]
    t0 = ip[3]#torch.tensor([torch.pi / 2], requires_grad=True)  #in_par[3]
    t1 = ip[4]#torch.tensor([0.0], requires_grad=True) #in_par[4]'''

    '''a = torch.empty(1).uniform_(0, torch.pi).requires_grad_()
    f0 = torch.empty(1).uniform_(0, 2 * torch.pi).requires_grad_()
    f1 = torch.empty(1).uniform_(0, 2 * torch.pi).requires_grad_()
    t0 = torch.empty(1).uniform_(0, 2 * torch.pi).requires_grad_()
    t1 = torch.empty(1).uniform_(0, 2 * torch.pi).requires_grad_()'''

    # Set learning rate
    #learning_rate = 0.001 + 1**(N/10000) #0.01
    
    #print(f'exp learning rate: {N/10000}')
    #print(f'learning rate: {learning_rate}')


    # Run gradient descent for each initial condition
    solutions = []
    for ip in tqdm(initial_conditions):

        # Create a list to store the loss at each epoch
        loss_history = []
        a_grad = []
        f0_grad = []
        f1_grad = []
        t0_grad = []
        t1_grad = []

        # Set the initial condition
        a, f0, f1, t0, t1 = ip
        # Train for N epochs
        for i in range(200):
            
            # Forward pass:
            loss = floss(nj_vec, a, f0, f1, t0, t1)
            # append the current loss to the history
            loss_history.append(loss.item())
            

            # Backward pass: compute gradients
            loss.backward()

            a_grad.append(a.grad.item())
            #a_grad.append((torch.sign(a.grad).item())*a.grad.item())
            f0_grad.append(f0.grad.item())
            #f0_grad.append((torch.sign(f0.grad).item())*f0.grad.item())
            f1_grad.append(f1.grad.item())
            #f1_grad.append((torch.sign(f1.grad).item())*f1.grad.item())
            t0_grad.append(t0.grad.item())
            #t0_grad.append((torch.sign(t0.grad).item())*t0.grad.item())
            t1_grad.append(t1.grad.item())
            #t1_grad.append((torch.sign(t1.grad).item())*t1.grad.item())

            # Update parameters using gradient descent
            with torch.no_grad():
                a -= learning_rate * a.grad
                f0 -= learning_rate * f0.grad
                f1 -= learning_rate * f1.grad
                t0 -= learning_rate * t0.grad
                t1 -= learning_rate * t1.grad

            # Zero gradients for the next iteration
            a.grad.zero_()
            f0.grad.zero_()
            f1.grad.zero_()
            t0.grad.zero_()
            t1.grad.zero_()

        # Store the final solution and its loss
        solutions.append((ip, loss.item()))

        # after the training loop
        plt.figure()
        plt.plot(loss_history, label='Loss')
        plt.plot(a_grad, label='a_grad')
        plt.plot(f0_grad, label='f0_grad')
        plt.plot(f1_grad, label='f1_grad')
        plt.plot(t0_grad, label='t0_grad')
        plt.plot(t1_grad, label='t1_grad')
        plt.xlabel('Epoch')
        plt.legend()
        plt.show()  

    # Find the solution with the lowest loss
    best_solution, best_loss = min(solutions, key=lambda solution: solution[1])
    #print(best_solution)

    a, f0, f1, t0, t1 = best_solution                                   
    
    p0 = (torch.cos(a))**2
    p1 = (torch.sin(a))**2
    #print('p0+p1:', p0.item()+p1.item())
    a0 = torch.cos(t0)
    a1 = torch.sin(t0)
    #print(f'a0^2 + a1^2: {(a0.item())**2 + (a1.item())**2}')
    b0 = torch.cos(t1)
    b1 = torch.sin(t1)
    #print(f'b0^2 + b1^2: {(b0.item())**2 + (b1.item())**2}')
    ph0 = torch.exp(1j*f0)
    ph1 = torch.exp(1j*f1)
    #print(f'p0: {p0.item()}, p1: {p1.item()}, a0: {a0.item()}, a1: {a1.item()}, b0: {b0.item()}, b1: {b1.item()}, ph0: {ph0.item()}, ph1: {ph1.item()}')
    
    psi0n = torch.tensor([a0, ph0*a1], dtype=torch.cdouble)
    psi1n = torch.tensor([b0, ph1*b1], dtype=torch.cdouble)
    initial_states = Creating_states(coeff = coeff, abstract=False)     # Creating the two states with these coefficients
    psi0 = initial_states[0]
    psi1 = initial_states[1]    # created the states to be discriminated
    psi0p = [cm.polar(psi0[0])[0], cm.polar(psi0[1])[0]*np.exp(1j*(cm.polar(psi0[1])[1]-cm.polar(psi0[0])[1]))]
    psi1p = [cm.polar(psi1[0])[0], cm.polar(psi1[1])[0]*np.exp(1j*(cm.polar(psi1[1])[1]-cm.polar(psi1[0])[1]))]
    psi0t = torch.tensor(psi0p, dtype=torch.cdouble)
    psi1t = torch.tensor(psi1p, dtype=torch.cdouble)
   
    fid0 = torch.abs(torch.dot(psi0n, torch.conj(psi0t)))**2
    fid1 = torch.abs(torch.dot(psi1n, torch.conj(psi1t)))**2

    fid = [fid0.item(), fid1.item()]
    p = [p0.item(), p1.item()]

    return([fid,p])

In [ ]:
coeff = [0**.5,1**.5,1**.5,0**.5] 
priors = [.5,.5]
trials = [100*(i+1) for i in range(100)]
p0trials  = [priors[0] for i in trials] 
p1trials  = [priors[1] for i in trials]

lr0 = 0.01
#ip0 = [torch.tensor([val], requires_grad=True) for val in [torch.pi / 4, 0.0, 0.0, torch.pi / 2, 0.0]]
#print(ip0)
#ip = [torch.tensor([torch.pi / 4], requires_grad=True),torch.tensor([0.0], requires_grad=True),torch.tensor([0.0], requires_grad=True),torch.tensor([torch.pi / 2], requires_grad=True),torch.tensor([0.0], requires_grad=True)]

initial_states = Creating_states(abstract=False, coeff = coeff)     # Creating the two states with these coefficients
psi0 = initial_states[0]
psi1 = initial_states[1]

output_f = []
output_p = []

pfid  = [1 for i in trials]    # perfect fidelity of 1
#for i,t in tqdm(enumerate(trials)):
    #print(f'i_N: {i}')
    #lr = update_lr(lr0,i)
    #ip = ip0
    #print(f'learning rate: {lr}')
    #output_f.append(grad_fid(lr, t,coeff,priors)[0])
    #output_p.append(grad_fid(lr, t,coeff,priors)[1])

output_f = [grad_fid(lr0, i, coeff, priors)[0] for i in tqdm(trials)]
output_p = [grad_fid(lr0, i, coeff, priors)[1] for i in tqdm(trials)]

outputf = list(map(list, zip(*output_f)))
outputp = list(map(list, zip(*output_p)))

#print(outputf)
#print(outputp)

In [ ]:
fig_a, (ax1a, ax2a) = plt.subplots(1,2, figsize=(10,4), sharey=True)
ax1a.plot(trials, outputf[0], label=r'$F_0=|\langle \psi_0|\psi_0^{\,n}\rangle|^2$')
ax1a.plot(trials , pfid, "--", label=r'$F=1$')
ax1a.set_ylim(0,1)
ax1a.set_xlabel(' N (trials)')
ax1a.set_ylabel(r'Fidelity $F$')
ax1a.set_title(r'$|\psi_0\rangle ={}|0\rangle+{}|1\rangle $, $p_0={}$'.format(roundc(coeff[0],1), roundc(coeff[1],1), priors[0]))
ax1a.legend(loc='best')
ax2a.plot(trials, outputf[1], label=r'$F_1=|\langle \psi_1|\psi_1^{\,n}\rangle|^2$')
ax2a.plot(trials , pfid, "--", label=r'$F=1$')
ax2a.set_ylim(0,1)
ax2a.set_xlabel(' N (trials)')
ax2a.set_ylabel(r'Fidelity $F$')
ax2a.set_title(r'$|\psi_1\rangle ={}|0\rangle+{}|1\rangle $, $p_1={}$'.format(roundc(coeff[2],1), roundc(coeff[3],1), priors[1]))
ax2a.legend(loc='best')

 
fig_c, (ax1c, ax2c) = plt.subplots(1,2, figsize=(10,4), sharey=True)
ax1c.plot(trials, outputp[0], label=r'$p_{0}$ numerical')
ax1c.plot(trials, p0trials, '--', label=r'$p_{0}$ theoretical')
ax2c.plot(trials, outputp[1], label=r'$p_{1}$ numerical')
ax2c.plot(trials, p1trials, '--', label=r'$p_{1}$ theoretical')
ax1c.set_ylim(0,1)
ax2c.set_ylim(0,1)
ax1c.set_xlabel(' N (trials)')
ax2c.set_xlabel(' N (trials)')
ax1c.set_ylabel(r'Probability')
ax2c.set_ylabel(r'Probability')
ax1c.legend(loc='best')
ax2c.legend(loc='best')
ax1c.set_title(r'$|\psi_0\rangle ={}|0\rangle+{}|1\rangle $, $p_0={}$'.format(roundc(coeff[0],1), roundc(coeff[1],1), priors[0]))
ax2c.set_title(r'$|\psi_1\rangle ={}|0\rangle+{}|1\rangle $, $p_1={}$'.format(roundc(coeff[2],1), roundc(coeff[3],1), priors[1]))

plt.show()